# Webscraping IMDB website 


In [2]:
import bs4
import pandas as pd
import requests
import numpy as np

In [3]:
url='https://www.imdb.com/search/title/?&genres=sci_fi'

Fetching the content from the website

In [4]:
result = requests.get(url)

Creating the soup from beautiful soup

In [5]:
soup = bs4.BeautifulSoup(result.text,'lxml')
soup

<!DOCTYPE html>
<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:'java'};</script>
<script>
    if (typeof uet == 'function') {
      uet("bb", "LoadTitle", {wb: 1});
    }
</script>
<script>(function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);</script>
<title>Top 50 Sci-Fi Movies and TV Shows - IMDb</title>
<script>(function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);</script>
<script>
    if (typeof uet == 'function') {
      uet("be", "LoadTitle", {wb: 1});
    }
</script>
<script>
    if (typeof uex == 'function') {
      uex("ld", "LoadTitle", {wb: 1});
    }
</script>
<link href="https://www.imdb.com/search/title/?genres=sci-fi" rel="canonical"/>
<meta content="http://www.imdb.com/search

In [6]:
sci_fi_container = soup.select('.lister-item-content')
sci_fi_container[0]

[<div class="lister-item-content">
 <h3 class="lister-item-header">
 <span class="lister-item-index unbold text-primary">1.</span>
 <a href="/title/tt9032400/">Eternals</a>
 <span class="lister-item-year text-muted unbold">(2021)</span>
 </h3>
 <p class="text-muted">
 <span class="certificate">UA</span>
 <span class="ghost">|</span>
 <span class="runtime">156 min</span>
 <span class="ghost">|</span>
 <span class="genre">
 Action, Adventure, Fantasy            </span>
 </p>
 <div class="ratings-bar">
 <div class="inline-block ratings-imdb-rating" data-value="6.9" name="ir">
 <span class="global-sprite rating-star imdb-rating"></span>
 <strong>6.9</strong>
 </div>
 <div class="inline-block ratings-user-rating">
 <span class="userRatingValue" data-tconst="tt9032400" id="urv_tt9032400">
 <span class="global-sprite rating-star no-rating"></span>
 <span class="rate" data-no-rating="Rate this" data-value="0" name="ur">Rate this</span>
 </span>
 <div class="starBarWidget" id="sb_tt9032400">
 <

To start with we scrape for 1 movie and scale it up with for loop


In [7]:
test_title = sci_fi_container[0].find('h3',class_='lister-item-header')
test_title.a.text

'Eternals'

In [8]:
test_year = sci_fi_container[0].find('span',class_='lister-item-year text-muted unbold')
test_year.text.replace(')','').replace('(','')# removing () 

'2021'

In [9]:
test_rating=sci_fi_container[0].find('div',class_='inline-block ratings-imdb-rating')
test_rating['data-value']

'6.9'

In [10]:
test_description = sci_fi_container[0].find_all('p',class_='text-muted')
test_description[1].text.replace("\n",'')

'The saga of the Eternals, a race of immortal beings who lived on Earth and shaped its history and civilizations.'

In [11]:
test_directior=sci_fi_container[46].find('p',class_='')
test_directior.text.replace('\n','').lstrip()

'Directors:Anthony Russo, Joe Russo|     Stars:Robert Downey Jr., Chris Evans, Mark Ruffalo, Chris Hemsworth'

In [12]:
test_directior=sci_fi_container[0].find('p',class_='')
string = test_directior.text.replace('\n','').lstrip() # remove \n and empty spaces
Stars_index = string.index('Stars')# index of Stars
Stars = string[Stars_index+6::]# slicing from stars
print(Stars) 
# print(string)
line_index=string.index('|')
Director=string[9:line_index]
print(Director)


Gemma Chan, Richard Madden, Angelina Jolie, Salma Hayek
Chloé Zhao


In [13]:
test_rating=sci_fi_container[0].find('span',class_='certificate')
test_rating.text

'UA'

In [14]:
test_runtime = sci_fi_container[0].find('span',class_='runtime')
test_runtime.text[:-4]

'156'

In [15]:
test_genre = sci_fi_container[0].find('span',class_='genre')
test_genre.text.replace('\n','').rstrip()

'Action, Adventure, Fantasy'

Now we have checked for a particular movie, let's make it for 50 movies and replace missing values with 'nan'(not a  number) 

In [16]:
#initialize empty lists to store the variables scraped
titles = []
imdb_rating = []
movie_ratings = []
cast = []
runtime = []
movie_genre = []
movie_director = []
description = []
movie_year = []

for movie in sci_fi_container:
    #Title
    title = movie.find('h3',class_='lister-item-header')
    titles.append(title.a.text)
    
    # Year
    year = movie.find('span',class_='lister-item-year text-muted unbold')
    movie_year.append(year.text.replace(')','').replace('(','').replace('– ',''))#removing '()' and '-'
    
    #movie rating
    # try and except block for handling null values
    try:
        rating = movie.find('span',class_='certificate')
        movie_ratings.append(rating.text)
    except:
        movie_ratings.append(np.nan)

    #genre
    genre = movie.find('span',class_='genre')
    movie_genre.append(genre.text.replace('\n','').rstrip())
    
    #runtime
    try:
        time = movie.find('span',class_='runtime')
        runtime.append(int(time.text[:-4]))
    except:
        runtime.append(np.nan)
        
    # movie director
    temp = movie.find('p',class_='')
    string = temp.text.replace('\n','').lstrip() # remove \n and empty spaces
    try:
        line_index=string.index('|')
        Director=string[9:line_index]
        movie_director.append(Director)
    except:
        movie_director.append(np.nan)
    
    #cast
    stars_index = string.index('Stars')# index of Stars
    stars = string[stars_index+6::]# slicing from stars
    cast.append(stars)
    
    #plot
    plot = movie.find_all('p',class_='text-muted')
    description.append(plot[1].text.replace("\n",''))
    
    #star rating
    try:
        imdb_score = movie.find('div',class_='inline-block ratings-imdb-rating')
        imdb_rating.append(imdb_score['data-value'])
    except:
        imdb_rating.append(np.nan)

Creating dataframe and adding movies into it


In [17]:
movies_df = pd.DataFrame({'movie':titles,'plot':description,'year':movie_year,'ratings':movie_ratings,
                          'stars':imdb_rating,'runtime':runtime,'genre':movie_genre,'director':movie_director,'cast':cast})
print(movies_df.info())
movies_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   movie     50 non-null     object 
 1   plot      50 non-null     object 
 2   year      50 non-null     object 
 3   ratings   43 non-null     object 
 4   stars     42 non-null     object 
 5   runtime   41 non-null     float64
 6   genre     50 non-null     object 
 7   director  21 non-null     object 
 8   cast      50 non-null     object 
dtypes: float64(1), object(8)
memory usage: 3.6+ KB
None


,movie,plot,year,ratings,stars,runtime,genre,director,cast
0,Eternals,"The saga of the Eternals, a race of immortal b...",2021,UA,6.9,156.0,"Action, Adventure, Fantasy",Chloé Zhao,"Gemma Chan, Richard Madden, Angelina Jolie, Sa..."
1,Arcane: League of Legends,Set in utopian Piltover and the oppressed unde...,2021,16,9.4,NaN,"Animation, Action, Adventure",NaN,"Kevin Alejandro, Jason Spisak, Hailee Steinfel..."
2,Dune: Part One,Feature adaptation of Frank Herbert's science ...,2021,UA,8.2,155.0,"Action, Adventure, Drama",Denis Villeneuve,"Timothée Chalamet, Rebecca Ferguson, Zendaya, ..."
3,Shang-Chi and the Legend of the Ten Rings,"Shang-Chi, the master of weaponry-based Kung F...",2021,UA,7.7,132.0,"Action, Adventure, Fantasy",Destin Daniel Cretton,"Simu Liu, Awkwafina, Tony Chiu-Wai Leung, Ben ..."
4,Finch,"On a post-apocalyptic earth, a robot, built to...",2021,UA 13+,7,115.0,"Adventure, Drama, Sci-Fi",Miguel Sapochnik,"Tom Hanks, Caleb Landry Jones, Marie Wagenman,..."
